# Assignment 2 - train.ipynb
Model version control and experiment tracking with MLflow.

In [1]:
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, average_precision_score

/Users/codula/Documents/CMI/AML/AppliedMachineLearning/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Setup

In [2]:
mlflow.set_tracking_uri('sqlite:///mlflow.db')
experiment_name = 'sms_spam_benchmarks'
mlflow.set_experiment(experiment_name)

train_df = pd.read_csv('train.csv')
validation_df = pd.read_csv('validation.csv')
test_df = pd.read_csv('test.csv')

print(train_df.shape, validation_df.shape, test_df.shape)

2026/02/17 00:07:24 INFO mlflow.store.db.utils: Creating initial MLflow database tables...


2026/02/17 00:07:24 INFO mlflow.store.db.utils: Updating database tables


INFO  [alembic.runtime.migration] Context impl SQLiteImpl.


INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step


...

INFO  [alembic.runtime.migration] Running upgrade 6953534de441 -> bda7b8c39065, increase_model_version_tag_value_limit


INFO  [alembic.runtime.migration] Context impl SQLiteImpl.


INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


2026/02/17 00:07:25 INFO mlflow.tracking.fluent: Experiment with name 'sms_spam_benchmarks' does not exist. Creating a new experiment.


(3900, 2) (836, 2) (836, 2)


## Helper Functions

In [3]:
def build_vectorizer():
    return TfidfVectorizer(stop_words='english', max_features=5000)


def build_model(model_name: str):
    registry = {
        'nb': MultinomialNB(),
        'lr': LogisticRegression(max_iter=1000, random_state=42),
        'svm': LinearSVC(max_iter=2000, random_state=42),
    }
    return registry[model_name]


def get_scores(model, X_vec, y_true):
    y_pred = model.predict(X_vec)

    if hasattr(model, 'predict_proba'):
        y_score = model.predict_proba(X_vec)[:, 1]
    elif hasattr(model, 'decision_function'):
        y_score = model.decision_function(X_vec)
    else:
        y_score = y_pred

    return {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred, zero_division=0),
        'recall': recall_score(y_true, y_pred, zero_division=0),
        'f1': f1_score(y_true, y_pred, zero_division=0),
        'aucpr': average_precision_score(y_true, y_score),
    }

## Build, Track, and Register 3 Benchmark Models

In [4]:
benchmark_models = ['nb', 'lr', 'svm']
results = []

for model_name in benchmark_models:
    with mlflow.start_run(run_name=model_name):
        vectorizer = build_vectorizer()

        X_train = vectorizer.fit_transform(train_df['message'])
        X_val = vectorizer.transform(validation_df['message'])
        X_test = vectorizer.transform(test_df['message'])

        y_train = train_df['target']
        y_val = validation_df['target']
        y_test = test_df['target']

        model = build_model(model_name)
        model.fit(X_train, y_train)

        val_metrics = get_scores(model, X_val, y_val)
        test_metrics = get_scores(model, X_test, y_test)

        mlflow.log_param('model_name', model_name)
        mlflow.log_param('vectorizer', 'tfidf_stopwords_english_max5000')
        mlflow.log_param('random_state', 42)

        mlflow.log_metrics({
            'validation_accuracy': val_metrics['accuracy'],
            'validation_precision': val_metrics['precision'],
            'validation_recall': val_metrics['recall'],
            'validation_f1': val_metrics['f1'],
            'validation_aucpr': val_metrics['aucpr'],
            'test_accuracy': test_metrics['accuracy'],
            'test_precision': test_metrics['precision'],
            'test_recall': test_metrics['recall'],
            'test_f1': test_metrics['f1'],
            'test_aucpr': test_metrics['aucpr'],
        })

        registered_model_name = f'sms_spam_{model_name}'
        mlflow.sklearn.log_model(
            sk_model=model,
            artifact_path='model',
            registered_model_name=registered_model_name,
        )

        results.append({
            'model': model_name,
            'validation_aucpr': val_metrics['aucpr'],
            'test_aucpr': test_metrics['aucpr'],
            'run_id': mlflow.active_run().info.run_id,
            'registered_model': registered_model_name,
        })

results_df = pd.DataFrame(results).sort_values('validation_aucpr', ascending=False).reset_index(drop=True)
print('Benchmark model selection metric (AUCPR):')
display(results_df[['model', 'validation_aucpr', 'test_aucpr']])

best_model = results_df.iloc[0]['model']
print(f'Best model selected using validation AUCPR: {best_model}')

2026/02/17 00:07:25 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


2026/02/17 00:07:26 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


2026/02/17 00:07:26 INFO mlflow.store.db.utils: Creating initial MLflow database tables...


2026/02/17 00:07:26 INFO mlflow.store.db.utils: Updating database tables


INFO  [alembic.runtime.migration] Context impl SQLiteImpl.


...

2026/02/17 00:07:28 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Benchmark model selection metric (AUCPR):


Successfully registered model 'sms_spam_svm'.
Created version '1' of model 'sms_spam_svm'.


,model,validation_aucpr,test_aucpr
0,svm,0.977774,0.990482
1,lr,0.972808,0.974854
2,nb,0.955947,0.968580


Best model selected using validation AUCPR: svm


## Checkout AUCPR from MLflow Runs

In [5]:
exp = mlflow.get_experiment_by_name(experiment_name)
runs = mlflow.search_runs(
    experiment_ids=[exp.experiment_id],
    order_by=['metrics.validation_aucpr DESC'],
)

view_cols = [
    'tags.mlflow.runName',
    'metrics.validation_aucpr',
    'metrics.test_aucpr',
    'run_id',
]
print('AUCPR per benchmark model from MLflow tracking:')
display(runs[view_cols])

AUCPR per benchmark model from MLflow tracking:


,tags.mlflow.runName,metrics.validation_aucpr,metrics.test_aucpr,run_id
0,svm,0.977774,0.990482,fdf323c36a5e4953b0901e8a753443d9
1,lr,0.972808,0.974854,74ed0562e83c410fb3c1b76f7ae8c2b8
2,nb,0.955947,0.968580,9532e19f05c248f7b7ac5e2e0847ca51
